In [5]:
import urllib.request, sys
from bs4 import BeautifulSoup
import requests
from time import sleep
%env https_proxy=http://172.16.2.250:3128

env: https_proxy=http://172.16.2.250:3128


In [38]:
def scrape_from_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    title = soup.find("h1", {"class" : "story-title"}).get_text()
    date = soup.time.attrs['datetime'][:10]
    summary = soup.find("h2", {"class" : "story-description"}).get_text()
    content = "\n".join([elt.text for elt in soup.find("div", {"class" : "story-container"}).find_all("p") if not ("READ:" in elt.text)])

    return [title, date, url, summary, content]

In [16]:
def get_topic_pages(page):
    soup = BeautifulSoup(page.content, "html.parser")
    urls = [elt.find("a")["href"] for elt in soup.find_all("article", {"class" : "channel-card"})]
    return(urls)

In [17]:
def scrape(page_no, category):
    URL = f"https://www.republicworld.com//technology-news/science/page-{page_no}"
    url_list = get_topic_pages(requests.get(URL))
    sleep(1)

    data = []

    for url in url_list:
        data.append(scrape_from_page(url))
        sleep(1)

    return data

In [ ]:
def scrape_loop(start_pg, end_pg, category, subcategory):
    data = []
    for pg in range(start_pg, end_pg):
        data += scrape(pg, category, subcategory)
        
    return data